In [ ]:

# Conversor BPA -> Plano de Ação
!pip install openpyxl pandas

import pandas as pd
import openpyxl
from openpyxl.drawing.image import Image
from openpyxl.styles import Alignment
from datetime import datetime
from google.colab import files

# ======= CONFIGURAÇÕES =======
URL_ARQUIVO_ENTRADA = "https://github.com/fabiaalves/conversor-planner-bahia/raw/main/BPA%20TESTE%20por%20produto.xlsx"
URL_MAPEAMENTO = "https://github.com/fabiaalves/conversor-planner-bahia/raw/main/Mapeamento_Colunas.xlsx"
URL_MODELO = "https://github.com/fabiaalves/conversor-planner-bahia/raw/main/PLANO%20DE%20A%C3%87%C3%83O%20-%20PROJETO%20BAHIA%20QUE%20PRODUZ%20E%20ALIMENTA.xlsx"
URL_IMAGEM = "https://github.com/fabiaalves/conversor-planner-bahia/raw/main/nova_imagem.png"  # imagem enviada

# ======= BAIXAR ARQUIVOS =======
import requests
def baixar_arquivo(url, nome_arquivo):
    r = requests.get(url)
    with open(nome_arquivo, 'wb') as f:
        f.write(r.content)

baixar_arquivo(URL_ARQUIVO_ENTRADA, "entrada.xlsx")
baixar_arquivo(URL_MAPEAMENTO, "mapeamento.xlsx")
baixar_arquivo(URL_MODELO, "modelo.xlsx")
baixar_arquivo(URL_IMAGEM, "imagem_topo.png")

# ======= PROCESSAMENTO =======
df_entrada = pd.read_excel("entrada.xlsx")
df_mapeamento = pd.read_excel("mapeamento.xlsx")

wb = openpyxl.load_workbook("modelo.xlsx")
ws = wb.active

# Substituir imagem do topo
for img in ws._images:
    ws._images.remove(img)
nova_img = Image("imagem_topo.png")
nova_img.anchor = 'A1'
ws.add_image(nova_img)

# Ler cabeçalhos do modelo
headers_modelo = [cell.value for cell in ws[1]]

# Preencher dados conforme mapeamento
for _, row in df_mapeamento.iterrows():
    origem = row.get('Coluna do Planner (origem)')
    destino = row.get('Coluna na nova planilha (destino)')
    regra = str(row.get('Transformação / Regra especial', '')).strip().lower()
    formato = row.get('Formato esperado', '')

    if pd.isna(destino) or destino not in headers_modelo:
        continue

    col_idx = headers_modelo.index(destino) + 1

    for i in range(len(df_entrada)):
        valor = ""
        if regra == "valor fixo":
            valor = formato
        elif regra == "data (dd/mm/yyyy)" and origem in df_entrada.columns:
            valor = pd.to_datetime(df_entrada.loc[i, origem], errors='coerce').strftime('%d/%m/%Y') if pd.notna(df_entrada.loc[i, origem]) else ""
        elif origem in df_entrada.columns:
            valor = df_entrada.loc[i, origem]
        ws.cell(row=i+2, column=col_idx, value=valor)

# Aplicar quebra de texto e ajuste de largura
for col in ws.columns:
    max_length = 0
    column = col[0].column_letter
    for cell in col:
        cell.alignment = Alignment(wrap_text=True, vertical='top')
        if cell.value:
            max_length = max(max_length, len(str(cell.value)))
    ws.column_dimensions[column].width = min(max_length + 2, 50)
for row in ws.iter_rows(min_row=2):
    ws.row_dimensions[row[0].row].height = 30

# Salvar arquivo final
hoje = datetime.now().strftime('%d-%m-%Y')
nome_saida = f"PLANO_DE_ACAO_FINAL_{hoje}.xlsx"
wb.save(nome_saida)

# Download automático
files.download(nome_saida)
